In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [ ]:
df_raw = pd.read_csv('/kaggle/input/heart-attack-data-analysis/Heart Attack Data Set.csv')

# Dicionário de dados do Dataset

**Dados Demográficos**

age: Idade do paciente (em anos).

sex: Sexo biológico do paciente (1 = Homem; 0 = Mulher).

**Sintomas e Exames Clínicos Simples**

cp (Chest Pain type): Tipo de dor no peito relatada.

0 = Angina típica (dor no peito clássica por falta de sangue no coração).

1 = Angina atípica (dor no peito, mas não com as características clássicas).

2 = Dor não anginosa (geralmente espasmos ou problemas em outros músculos/órgãos).

3 = Assintomático (sem dor).

trestbps (Resting Blood Pressure): Pressão arterial do paciente em repouso (em mm Hg).

chol: Colesterol sérico (em mg/dl).

fbs (Fasting Blood Sugar): Açúcar no sangue em jejum. Se for maior que 120 mg/dl, recebe o valor 1 (Verdadeiro). Se for menor, recebe 0 (Falso). É um indicativo para diabetes.

**Resultados de Eletrocardiograma (ECG) e Testes de Esforço**

restecg (Resting ECG): Resultados do eletrocardiograma em repouso.

0 = Normal.

1 = Anormalidade da onda ST-T (pode indicar problemas na forma como o coração relaxa após bater).

2 = Provável hipertrofia ventricular esquerda (o músculo do coração está aumentado/espesso).

thalach (Maximum Heart Rate): Frequência cardíaca (batimentos por minuto) máxima atingida pelo paciente durante um teste de esforço físico na esteira.

exang (Exercise Induced Angina): Angina (dor no peito) induzida por exercício físico (1 = Sim; 0 = Não).

oldpeak: Uma medida no eletrocardiograma chamada "Depressão do segmento ST". Ela mede a diferença no gráfico do coração entre o momento de maior esforço e o momento de descanso.

slope (Slope): A inclinação do segmento ST no gráfico do eletrocardiograma durante o pico do exercício. Pode ser: 0 = Descendente, 1 = Plano, 2 = Ascendente.

**Exames de Imagem**

ca (Number of major vessels): Número de vasos sanguíneos principais (de 0 a 3) que ficaram visíveis após serem coloridos por um exame chamado fluoroscopia. Se o vaso aparece bem colorido, significa que não está entupido e o sangue está fluindo bem.

thal: Resultado do teste de estresse com tálio (um tipo de exame de imagem que avalia o fluxo de sangue no músculo do coração).

1 = Defeito fixo (o sangue não chega em alguma parte do coração, indicando possível cicatriz de infarto anterior).

2 = Normal (fluxo sanguíneo normal).

3 = Defeito reversível (o fluxo de sangue é normal em repouso, mas diminui quando o coração faz esforço).

**O Diagnóstico**

target: É a conclusão do diagnóstico médico.

0 = Menor chance de ter um ataque cardíaco (coração mais saudável).

1 = Maior chance de ter um ataque cardíaco (presença de doença cardíaca).

In [ ]:
df_raw.head()

In [ ]:
df_raw.shape

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

# **> Limpeza e Organização dos dados**

**Renomeando as colunas:**

In [ ]:
df = df_raw.copy()

In [ ]:
novos_nomes = {
    'age': 'idade',
    'sex': 'sexo',
    'cp': 'tipo_dor_peito',
    'trestbps': 'pressao_arterial_repouso',
    'chol': 'colesterol',
    'fbs': 'acucar_sangue_jejum',
    'restecg': 'ecg_repouso',
    'thalach': 'freq_cardiaca_max',
    'exang': 'angina_exercicio',
    'oldpeak': 'depressao_st',
    'slope': 'inclinacao_st',
    'ca': 'num_vasos_principais',
    'thal': 'resultado_talio',
    'target': 'risco_ataque_cardiaco'
}

#Nomes traduzidos e sugeridos pelo Gemini

In [ ]:
df.rename(columns=novos_nomes, inplace=True)

In [ ]:
print(df.columns)

In [ ]:
df.head()

**Substituindo valores categóricos:**

In [ ]:
df['sexo'] = df['sexo'].replace({1: 'Homem', 0: 'Mulher'})

In [ ]:
dicionario_dor = {
    0: 'Angina Típica',
    1: 'Angina Atípica',
    2: 'Não Anginosa',
    3: 'Assintomático'
}

#Nomes traduzidos e sugeridos pelo Gemini

In [ ]:
df['tipo_dor_peito'] = df['tipo_dor_peito'].replace(dicionario_dor) 

In [ ]:
dicionario_risco_ataque_cardiaco = {
    0: 'Menor Risco',
    1: 'Maior Risco'
}

In [ ]:
df['risco_ataque_cardiaco'] = df['risco_ataque_cardiaco'].replace(dicionario_risco_ataque_cardiaco)

In [ ]:
print(df[['sexo','tipo_dor_peito', 'risco_ataque_cardiaco']].head())

# > **Análise Exploratória Demográfica**

**Qual é o tamanho do nosso grupo de estudo?**

In [ ]:
sns.set_theme(style="whitegrid")

In [ ]:
qtd_pacientes = df.shape[0]
qtd_variaveis = df.shape[1]

print(f"O dataset contém dados de {qtd_pacientes} pacientes.")
print(f"Temos {qtd_variaveis} informações diferentes para cada paciente.")

**Qual é a distribuição por Sexo?**

In [ ]:
contagem_sexo = df['sexo'].value_counts()
print('\nQuantidade de pacientes por sexo:')
print(contagem_sexo)

porcentagem_sexo = df['sexo'].value_counts(normalize=True) * 100
print('\nPorcentagem de pacientes por sexo (%):')
print(porcentagem_sexo.round(2))

In [ ]:
plt.figure(figsize=(9, 8))
ax = sns.countplot(x='sexo', data=df, palette='Set2')

plt.title('Distribuição de Pacientes por Sexo', fontsize=14, pad=15)
plt.xlabel('Sexo', fontsize=12)
plt.ylabel('Quantidade de Pacientes', fontsize=12)

for container in ax.containers:
    ax.bar_label(container, padding=8, fontsize=11)

plt.show()

**Qual é o perfil de Idade dos pacientes?**

In [ ]:
resumo_idade = df['idade'].describe()
print('\nResumo das idades dos pacientes:')
print(resumo_idade)

idade_min = df['idade'].min()
idade_max = df['idade'].max()
idade_med = df['idade'].mean()


print(f'\nO paciente mais novo tem {idade_min} anos.')
print(f'\nO paciente mais velho tem {idade_max} anos.')
print(f'\nA média de idade do grupo é de {idade_med:.1f} anos.')

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(data=df, x='idade', bins=15, kde=True, color='skyblue')

plt.title('Distribuição de Idade dos Pacientes', fontsize=14, pad=15)
plt.xlabel('Idade(Anos)', fontsize=12)
plt.ylabel('Frequência', fontsize=12)

plt.show()

**Distribuição de idade separada por sexo**

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(data=df, x='idade', hue='sexo', multiple='stack', bins=15, palette='Set2')

plt.title('Distribuição de Idade Separada por Sexo', fontsize=14)
plt.xlabel('Idade (Anos)')
plt.ylabel('Quantidade')

plt.show()


Perguntas que me surgiram durante a análise demográfica:
1. A quantidade de homens presente nesse dataset é muito maior que de mulheres.
2. A faixa etária maior varia de 50 á 70 anos.

* Homens infartam mais que mulheres? Ou o hospital que coletou esses dados simplesmente atende mais homens nessa faixa etária?
* A idade avançada é o maior fator?

# **Análise Bivariada**

Como a quantidade de homens é muito maior que a de mulheres, é natural que, em números absolutos, vejamos mais homens com risco de ataque cardíado. Por isso precisamos olhar para as proporções (porcentagens) dentro de cada grupo.

**Risco de ataque cardíaco de acordo com o sexo**

In [ ]:
risco_por_sexo = pd.crosstab(df['sexo'], df['risco_ataque_cardiaco'], normalize='index') * 100
print('Porcentagem de Risco dentro de cada Sexo:')
print(risco_por_sexo.round(2))

In [ ]:
ax = risco_por_sexo.plot(kind='bar', stacked=True, color=['#d62728', '#2ca02c'])

plt.title('Risco de ataque cardíado por sexo', fontsize=14, pad=15)
plt.xlabel('Sexo', fontsize=12)
plt.ylabel('Porcentagem (%)', fontsize=12)

plt.xticks(rotation=0)

plt.legend(title='Diagnóstico', loc='upper right', bbox_to_anchor=(1.3, 1))

plt.tight_layout()

plt.show()

**Resolução das dúvidas que apresentei acima:**
Apesar do número de homens nesse dataset ser muito maior, ao isolar cada grupo, percebemos que a porcentagem de risco é muito maior no grupo de mulheres (cerca de 75%).

**Viés de Amostragem (Sample Bias).**
Esses dados refletem a realidade apresentado nesse hospital em específico, mas que não significa necessariamente que mulheres têm mais ataques cardíacos que homens na população mundial.



**Risco de ataque cardíado de acordo com a idade**

In [ ]:
faixas = [20,40,50,60,80]
labels_faixas = ['Até 40 anos', '41-50 anos', '51-60 anos', 'Mais de 60 anos']

df['faixa_etaria'] = pd.cut(df['idade'], bins=faixas, labels=labels_faixas)

print('\nQuantidade de pacientes por faixa etária:')
print(df['faixa_etaria'].value_counts().sort_index())
print("-" * 40)

risco_por_idade = pd.crosstab(df['faixa_etaria'], df['risco_ataque_cardiaco'], normalize='index') * 100
print('\nPorcentagem de Risco por Faixa Etária:')
print(risco_por_idade.round(2))


In [ ]:
ax = risco_por_idade.plot(kind='bar', stacked=True, color=['#d62728', '#2ca02c'])

plt.title('Risco de ataque cardíado por idade', fontsize=14, pad=15)
plt.xlabel('Faixas de Idade', fontsize=12)
plt.ylabel('Porcentagem (%)', fontsize=12)

plt.xticks(rotation=0)

plt.legend(title='Diagnóstico', loc='upper right', bbox_to_anchor=(1.3, 1))

plt.tight_layout()

plt.show()

**Observação Demográfica:** O histograma revela que a maior concentração de pacientes testados neste dataset (129 pessoas) encontra-se na faixa de 51 a 60 anos. Isso indica que esta é a faixa etária com maior volume de dados coletados pelo hospital, refletindo provavelmente a idade em que check-ups cardiológicos e exames de esforço se tornam mais frequentes na rotina médica, e não necessariamente a faixa de maior risco percentual.

**Conclusão sobre Idade vs. Risco:** Ao analisar as proporções, notamos uma tendência contraintuitiva: as faixas etárias mais jovens (Até 40 anos e 41-50 anos) apresentam o maior percentual de diagnóstico de alto risco. Isso ocorre devido ao viés de seleção da amostra. Um paciente de 45 anos geralmente não realiza exames cardiológicos complexos (como testes de esforço e angiografia) apenas por rotina; ele geralmente é encaminhado porque já apresenta sintomas graves prévios. Já pacientes acima de 60 anos realizam esses exames frequentemente por precaução ou rotina, o que aumenta o número de diagnósticos negativos (Menor Risco) nessa faixa mais velha.

In [ ]:
correlacao_idade_risco = df_raw['age'].corr(df_raw['target'])
print(f"\nA correlação matemática entre Idade e Risco é: {correlacao_idade_risco:.2f}")

colunas_analise = ['age', 'trestbps', 'chol', 'thalach', 'target']

matriz_corr = df_raw[colunas_analise].corr()

plt.figure(figsize=(10,8))

sns.heatmap(matriz_corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, vmin=-1, vmax=1)

plt.title('Matriz de Correlação: Sinais Vitais e Risco (Dados Brutos)', fontsize=14, pad=15)

plt.show()

age = idade | trestbps = pressao | chol = colesterol | thalach = freq cardiaca max | target = risco

# Análise de acordo com os sintomas

Ao analisar o mapa de calor, vemos que o colesterol e o risco de ataque cardíaco não possuem correlação, vamos observar isso em um gráfico Boxplot:

In [ ]:
sns.boxplot(y='colesterol', x='risco_ataque_cardiaco', data=df, palette='Set2')

plt.title('Distribuição de Colesterol por Diagnóstico de Risco', fontsize=14, pad=15)
plt.xlabel('Diagnóstico', fontsize=12)
plt.ylabel('Nível de Colesterol (mg/dl)', fontsize=12)

plt.show()

O Boxplot confirma a ausência de correlação linear (Pearson = -0.09). A distribuição interquartil e as medianas de colesterol entre os grupos de maior e menor risco são visualmente sobreponíveis, indicando que este indicador, de forma isolada, não possui forte poder preditivo para esta amostra clínica.